In [1]:
import pandas as pd
df=pd.read_csv('D:/Study/INTERNSHIP/ETL/external/final_train.csv')

In [ ]:
last_row_file = 'last_row.txt'
num_batches = int(input("Enter the number of batches to send: "))

batch_size = 10000

# Read the last processed row from the text file1
try:
    with open(last_row_file, 'r') as f:
        last_processed_row = int(f.read())
except:
    last_processed_row = 0

result_df = pd.DataFrame()

for i in range(num_batches):
    chunk = next(pd.read_csv('D:/Study/INTERNSHIP/ETL/external/final_train.csv', skiprows=range(1, last_processed_row + 1), chunksize=batch_size), None)
    if chunk is not None:
        # Append the chunk to the result DataFrame
        result_df = pd.concat([result_df, chunk], ignore_index=True)
        last_processed_row += len(chunk)
        with open(last_row_file, 'w') as f:
            f.write(str(last_processed_row))
print(result_df.size)
result_df.tail()

In [35]:
df.head()

,id,week,center_id,city_code,region_code,center_type,op_area,meal_id,category,cuisine,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,136.83,152.29,0,0,177
1,1018704,2,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,135.83,152.29,0,0,323
2,1196273,3,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,132.92,133.92,0,0,96
3,1116527,4,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,135.86,134.86,0,0,163
4,1343872,5,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,146.50,147.50,0,0,215


In [36]:
df.shape

(456548, 15)

In [37]:
from confluent_kafka import Producer

In [59]:
topic = 'bigd'
bootstrap_servers = 'localhost:9092'
import logging
import os

In [60]:
conf = {'bootstrap.servers': bootstrap_servers,'acks': 'all',  
    'compression.type': 'gzip',
       "queue.buffering.max.messages": 10000000,}
producer = Producer(conf)

In [61]:
for index, row in df.iterrows():
    message = ','.join(map(str, row))
    producer.produce(topic, value=message)
producer.flush()

0